In [22]:
import tushare as ts
import os
import sys
import pandas as pd
import numpy as np

CONFIG_FILE_PATH = os.path.join(os.getcwd(), '../common')
sys.path.append(CONFIG_FILE_PATH)
from config import TOKEN

In [15]:
import chinese_calendar
import datetime
def get_holidays(year=None, include_weekends=True):
    """
    获取某一年的所有节假日，默认当年
    :param year: which year
    :param include_weekends: False for excluding Saturdays and Sundays
    :return: list
    """
    if not year:
        year = datetime.datetime.now().year
    start = datetime.date(year, 1, 1)
    end = datetime.date(year, 12, 31)
    holidays = chinese_calendar.get_holidays(start, end, include_weekends)
    return holidays

In [20]:
get_holidays(2022, False)

[datetime.date(2022, 1, 1),
 datetime.date(2022, 1, 2),
 datetime.date(2022, 1, 3),
 datetime.date(2022, 1, 31),
 datetime.date(2022, 2, 1),
 datetime.date(2022, 2, 2),
 datetime.date(2022, 2, 3),
 datetime.date(2022, 2, 4),
 datetime.date(2022, 2, 5),
 datetime.date(2022, 2, 6),
 datetime.date(2022, 4, 3),
 datetime.date(2022, 4, 4),
 datetime.date(2022, 4, 5),
 datetime.date(2022, 4, 30),
 datetime.date(2022, 5, 1),
 datetime.date(2022, 5, 2),
 datetime.date(2022, 5, 3),
 datetime.date(2022, 5, 4),
 datetime.date(2022, 6, 3),
 datetime.date(2022, 6, 4),
 datetime.date(2022, 6, 5),
 datetime.date(2022, 9, 10),
 datetime.date(2022, 9, 11),
 datetime.date(2022, 9, 12),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 10, 2),
 datetime.date(2022, 10, 3),
 datetime.date(2022, 10, 4),
 datetime.date(2022, 10, 5),
 datetime.date(2022, 10, 6),
 datetime.date(2022, 10, 7)]

In [23]:
pro = ts.pro_api(TOKEN)
print(TOKEN)

64ce1845b91d06f579525db6e53d497b1c513174331f5509320f4bd5


In [27]:

df = pro.index_basic(market='')

In [29]:
df

,ts_code,name,market,publisher,category,base_date,base_point,list_date
0,000001.CZC,农期指数,OTH,郑州商品交易所,商品指数,None,1000.0,None
1,000001.SH,上证指数,SSE,中证公司,综合指数,19901219,100.0,19910715
2,000002.CZC,农期谷物,OTH,郑州商品交易所,商品指数,None,1000.0,None
3,000002.SH,上证A指,SSE,中证公司,综合指数,19901219,100.0,19920221
4,000003.CZC,农期软商,OTH,郑州商品交易所,商品指数,None,1000.0,None
...,...,...,...,...,...,...,...,...
7995,h20811.CSI,中国1000材料(全)USD,CSI,中证公司,行业指数,20061229,1000.0,20160510
7996,h20812.CSI,中国1000工业(全)USD,CSI,中证公司,行业指数,20061229,1000.0,20160510
7997,h20813.CSI,中国1000可选(全)USD,CSI,中证公司,行业指数,20061229,1000.0,20160510
7998,h20814.CSI,中国1000消费(全)USD,CSI,中证公司,行业指数,20061229,1000.0,20160510


In [3]:
def get_price_money(date):
    # 日线行情 - 股票交易数据
    df_price = pro.daily(trade_date=date)
    print("股票交易数据: ", df_price.shape)
    
    # 日线行情 - 股票资金流向
    df_money = pro.moneyflow(trade_date=date)
    print("股票资金流向: ", df_money.shape)
    
    df = pd.merge(df_price, df_money, how = 'outer', on = ['ts_code', 'trade_date'])
    return df

In [4]:
df_20220907 = get_price_money('20220907')

股票交易数据:  (4898, 11)
股票资金流向:  (4787, 20)


In [6]:
df_20220907.shape

(4898, 29)

In [7]:
df_20220907.head(2)

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,...,buy_lg_vol,buy_lg_amount,sell_lg_vol,sell_lg_amount,buy_elg_vol,buy_elg_amount,sell_elg_vol,sell_elg_amount,net_mf_vol,net_mf_amount
0,000001.SZ,20220907,12.42,12.42,12.28,12.33,12.50,-0.17,-1.3600,979812.81,...,253009.0,31203.45,317818.0,39194.35,137847.0,16988.37,281030.0,34644.30,-340408.0,-41963.92
1,000002.SZ,20220907,17.66,17.82,17.40,17.52,17.85,-0.33,-1.8487,883386.10,...,268057.0,47065.42,267690.0,46955.95,131233.0,23019.18,147775.0,25908.38,-47445.0,-8222.11


In [60]:
# 同花顺概念和行业指数
df_industry = pro.ths_index()
print(df_industry.columns, df_industry.shape)
df_industry.head(2)

Index(['ts_code', 'name', 'count', 'exchange', 'list_date', 'type'], dtype='object') (1653, 6)


,ts_code,name,count,exchange,list_date,type
0,864005.TI,区块链,25.0,A,20200102,N
1,864006.TI,固态电池,3.0,A,20200102,N


In [70]:
df_industry.sort_values('count').head(2)

,ts_code,name,count,exchange,list_date,type
1300,877101.TI,烟草,1.0,HK,20150101,I
1025,861153.TI,海港与服务,1.0,US,None,I


In [63]:
df_industry['ts_code'].nunique()

1653

In [71]:
# 
df = pro.ths_daily(ts_code='877101.TI', 
                   start_date='20220907', end_date='20220907', fields='ts_code,trade_date,open,close,high,low,pct_change')

In [73]:
df

,ts_code,trade_date,close,open,high,low,pct_change


In [74]:
# 
df = pro.ths_daily(start_date='20220907', end_date='20220907', fields='ts_code,trade_date,open,close,high,low,pct_change')

In [75]:
df.shape

(1467, 7)

In [66]:
df['ts_code'].nunique()

1054

In [33]:
start_date, end_date = '20220101', '20220907'
ts_code = '600096.SH'


In [38]:
df_price_all = pro.daily(trade_date=end_date)

In [40]:
df_price_all.head(2)


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20220907,12.42,12.42,12.28,12.33,12.50,-0.17,-1.3600,979812.81,1208329.759
1,000002.SZ,20220907,17.66,17.82,17.40,17.52,17.85,-0.33,-1.8487,883386.10,1550325.356


In [43]:
df_price_all.sort_values('amount', ascending=False).head(10)

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
2209,300750.SZ,20220907,451.49,476.49,450.00,473.92,455.20,18.72,4.1125,216777.12,1.009527e+07
1072,002594.SZ,20220907,272.00,285.05,270.60,282.70,272.94,9.76,3.5759,190873.17,5.345728e+06
3021,600438.SH,20220907,56.00,58.02,55.98,56.69,56.87,-0.18,-0.3165,919203.91,5.238977e+06
3079,600519.SH,20220907,1835.03,1837.66,1811.01,1818.01,1845.00,-26.99,-1.4629,28306.16,5.151537e+06
948,002466.SZ,20220907,109.00,110.88,108.26,109.51,110.77,-1.26,-1.1375,390896.42,4.274902e+06
3476,601012.SH,20220907,53.00,54.28,52.71,53.46,53.16,0.30,0.5643,713985.67,3.842027e+06
1749,300274.SZ,20220907,121.98,126.60,121.10,122.16,122.59,-0.43,-0.3508,274861.51,3.405456e+06
1502,300014.SZ,20220907,94.00,99.60,93.36,98.49,94.80,3.69,3.8924,318640.79,3.107963e+06
856,002371.SZ,20220907,295.98,316.60,295.01,315.00,290.00,25.00,8.6207,101340.40,3.105520e+06
568,002077.SZ,20220907,16.00,17.66,15.87,17.66,16.05,1.61,10.0312,1627182.29,2.819322e+06


In [27]:
# 日线行情 - 获取单个股票交易数据
# df = pro.daily(trade_date='20180810')
df_price = pro.daily(ts_code=ts_code, start_date=start_date, end_date=end_date)
print(df_price.columns, df_price.shape)

Index(['ts_code', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close',
       'change', 'pct_chg', 'vol', 'amount'],
      dtype='object') (166, 11)


In [28]:
# 个股资金流向 - 获取单个股票交易数据
df_trade = pro.moneyflow(ts_code=ts_code, start_date=start_date, end_date=end_date)
print(df_trade.columns, df_trade.shape)

Index(['ts_code', 'trade_date', 'buy_sm_vol', 'buy_sm_amount', 'sell_sm_vol',
       'sell_sm_amount', 'buy_md_vol', 'buy_md_amount', 'sell_md_vol',
       'sell_md_amount', 'buy_lg_vol', 'buy_lg_amount', 'sell_lg_vol',
       'sell_lg_amount', 'buy_elg_vol', 'buy_elg_amount', 'sell_elg_vol',
       'sell_elg_amount', 'net_mf_vol', 'net_mf_amount'],
      dtype='object') (166, 20)


In [29]:
df = pd.merge(df_price, df_trade, how = 'outer', on = ['ts_code', 'trade_date'])

In [30]:
df.shape

(166, 29)

In [32]:
df[['trade_date', 'vol', 'amount', 'net_mf_vol', 'net_mf_amount']].head()

,trade_date,vol,amount,net_mf_vol,net_mf_amount
0,20220907,518811.74,1351711.454,-28218,-7217.54
1,20220906,906807.09,2375990.965,170965,44808.18
2,20220905,594687.23,1454065.589,62140,15432.26
3,20220902,650055.50,1624532.303,-143712,-35905.69
4,20220901,401049.07,1044564.477,-117133,-30434.12


In [17]:
df.tail()

,ts_code,trade_date,buy_sm_vol,buy_sm_amount,sell_sm_vol,sell_sm_amount,buy_md_vol,buy_md_amount,sell_md_vol,sell_md_amount,buy_lg_vol,buy_lg_amount,sell_lg_vol,sell_lg_amount,buy_elg_vol,buy_elg_amount,sell_elg_vol,sell_elg_amount,net_mf_vol,net_mf_amount
157,600096.SH,20220110,171057,31652.57,167801,31158.94,183072,33868.65,182960,33856.71,116421,21554.52,107504,19832.79,16106,2995.38,28390,5222.68,16216,3207.25
158,600096.SH,20220107,184676,35017.02,136438,25884.78,129571,24573.33,125081,23713.90,66781,12678.35,90180,17095.39,14098,2687.83,43427,8262.46,-77201,-14557.85
159,600096.SH,20220106,166838,32061.25,182223,35050.13,156900,30131.20,168811,32437.05,95499,18355.48,79616,15302.15,21228,4119.97,9814,1878.57,86931,16789.16
160,600096.SH,20220105,166863,31701.32,132610,25210.83,119623,22737.46,126584,24075.15,70808,13456.44,85560,16265.11,28658,5470.52,41198,7814.66,-93691,-17734.56
161,600096.SH,20220104,199176,38465.86,170836,33090.94,190236,36794.90,183913,35552.50,121345,23517.01,133501,25816.51,19431,3771.18,41938,8089.00,-11252,-2072.21


In [ ]:
# 每日涨跌停价格
#获取单日全部股票数据涨跌停价格
# df = pro.stk_limit(trade_date='20190625')

#获取单个股票数据
# df = pro.stk_limit(ts_code='002149.SZ', start_date='20190115', end_date='20190615')

In [34]:
df_most = pro.stk_limit(trade_date=end_date)

In [35]:
df_most.columns, df_most.shape

(Index(['trade_date', 'ts_code', 'up_limit', 'down_limit'], dtype='object'),
 (5800, 4))

In [36]:
df_most

,trade_date,ts_code,up_limit,down_limit
0,20220907,000001.SZ,13.75,11.25
1,20220907,000002.SZ,19.64,16.07
2,20220907,000004.SZ,9.87,8.93
3,20220907,000005.SZ,1.96,1.78
4,20220907,000006.SZ,4.92,4.02
...,...,...,...,...
5795,20220907,688295.SH,51.96,34.64
5796,20220907,688296.SH,27.06,18.04
5797,20220907,688297.SH,58.50,39.00
5798,20220907,688298.SH,107.03,71.35
